<pre>
목차
Attention Is All You Need 리뷰	
1. Introduction	
2. Background	
3. 제안 방법론	
3.2.3 Position-wise Feed Forward Networks	
3.2.4 Positional Encoding	
3.3 Contribution	
4 실험 및 결과	
4.1 Dataset	
4.2 Hardware and Schedule	
4.3 Optimizer	
4.4 Regularization	
4.6. Results	
4.7. Conclusion	

Language Models are Unsupervised Multitask Learners 리뷰	
1. Introduction	
1. 1 논문이 다루는 task 와 기존 연구	
2. Approach	
2.1 Training Dataset	
2.2 Input Representation	
2.3 Model	
3. 실험 및 결과	
3.1 Language Modeling	
3.2 Children’s Book Test	
3.4. Winograd Schema Challenge	
3.5 SOTA를 달성하지 못한 task	
3.6 Generalization & Memorization	
4. Conclusion	
참고문헌	
</pre>



# Attention Is All You Need 리뷰

<pre>
“Attention Is All You Need”는 Transformer를 발표하며 Recurrent 모델위주의 언어 모델에서 self-attention이라는 개념을 통해 자연어 처리에 큰 반향을 일으켰다. 
지금부터 해당 논문을 요약하며 Transformer를 파악해보려 한다.

1. Introduction
논문이 다루는 task
Abstract부터 나오듯이 해당 논문이 다루는 주제는 sequence transduction model로 이의 예시로는 기존 문장들을 기반으로 다음 단어를 예측하는 language model가 있다. 
따라서 해당 논문의 연구를 통해 English to German, English to French 기계번역에서 논문 시점에서 SOTA를 달성하였다.

기존 연구의 한계점
해당 연구 이전에 대부분의 언어모델과 기계 번역은 RNN과 LSTM등 반복적, 순차적 계산을 이용하는 모델이 대부분이었다. 
여러 연구에서 계속된 발전을 있었지만 sequential computation에서 발생하는 문제는 여전히 근본적인 문제점으로 남아있었고, 해당 논문에서는 이러한 방법에 대한 한계를 지적하고 해결방안을 제시하였다.

1. 병렬연산의 한계
RNN과 LSTM 은 이전의 결과값을 다음의 계산에 활용하기 때문에 계산이 순차적으로 일어난다. 
하지만 딥러닝이 발전할 수 있었던 이유인 GPU를 통한 병렬 계산이 순차적인 모델에서는 불가능하기 때문에 recurrent network 구조의 한계가 나타났다.
2. Long-term dependency
언어 모델에서 다루는 데이터는 문장의 형태로 존재한다. 
기존의 방법에서는 문장의 단어를 하나씩 입력 받아 그 다음 단어를 예측하기 때문에 초기에 단어에 대한 데이터가 문장의 길이가 길어질수록 초기의 데이터가 소실될 수 있다.
2. Background
순차 구조로 인한 계산량을 줄이고자 많은 연구들이 계속해서 제안되었다.
CNN을 활용하여 이를 극복하고자 하는 연구들도 제안되었으나 Long-term dependency 즉 거리가 먼 단어들 간의 연산이 더욱 어려워지는 결과를 초래하기도 하였다.
따라서 attention만을 활용하여 기존의 문제점을 해결하는 Transformer를 해당 논문에서 제안하였다.
3. 제안 방법론
3.1 Main Idea
해당 논문의 중요한 개념을 한문장으로 요약하자면 “Transformer는 recurrence를 피하며 attention만을 사용하여 더 빠르고 정확한 방법을 제공한다” 로 할 수 있다.
Transformer 는 크게 가지의 개념으로 구성되어 있다. Encoder and Decoder Stacks, Attention, Multi-Head Attention. 
그림 1-1은 이 개념이 포함된 전체 구조의 시각화이다.
그림에서 볼 수 있듯이 Encoder – Decoder 의 구조를 사용하고 병렬계산을 활용하고자 순차구조 대신 attention만에 의존하고 있다.

다음에서 각각의 내용을 상세하게 들여다보겠다.
3.2 Model Architecture
3.2.1 Encoder and decoder stack
Encoder 그리고 Decoder 모두 6개의 동일한 layer로 이루어져 있다. 


Encoder의 경우 Multi-head self attention mechanism 과 feed-forward network 로 하위 layer 2개를 이루고 있다. 
Decoder의 경우 Encoder의 하위 layer에서 Multi-head self attention mechanism 가 추가된 형태이다. 
Feed Forward network에서는 input과 output을 연결하여 정확도가 높은 가중치를 찾아 성능을 높이는 역할을 수행한다. 
layer 사이의 연결에서는 residual connection을 사용하여 연산 중에 데이터 손실을 막고 있음과 동시에 통해 특정 데이터에 치우치지 않도록 정규화를 진행하고 있다. 
각각의 layer에 사용되는 attention 별 설명은 다음 장에서 더욱 상세하게 설명하겠다.

3.2.2 Attention
Attention은 Querry와 비슷한 값은 가진 Key를 찾아서 Value를 얻는 과정이다. 
Attention의 개념은 해당 논문에 새롭게 제시된 것이 아니라 seq2seq모델등에서 이미 활용되고 있던 방법이다.
RNN과 같은 순환 구조와 함께 Attention을 사용할 경우 해당 논문이 지적한 한계인 병목현상, 장기 종속성 문제, 병렬처리 등의 어려움으로 Transformer 에서는 Attention 만을 활용한다.
Attention(Q,K,V)=softmax((QK^T)/√(d_k ))V

Transformer에서 Attention을 사용할 때 Scaled Dot-Product Attention을 사용한다. 위의 식에서의 기호의 의미는 아래의 표와 같다.
Q	현재 단어에 대한 임배딩 벡터
K	다른 단어들과 매칭을 위한 Key
V	Key와 매핑되어 있는 실제 단어를 나타내는 벡터
이 과정을 통해서 self-attention을 통해서 유사도를 추출하고 이를 통해 softamx를 이용하여 확률 분포를 추출한다. 
Scaled Dot-Product attention은 기존의 dot product attention에 스케일을 위해 Key 벡터 차원에 루트를 씌운 값으로 나누는 과정을 추가한 것이다. 

Multi-Head Attention
기계학습의 가장 간단한 형태인 Decision Tree를 사용하는 것 보다 여러 개의 Decision Tree를 사용한 Random Forest를 사용하는 경우 더 좋은 결과를 도출 할 수 있다. 
이는 데이터의 과적합을 방지하며 각각의 Decision Tree에서의 분류를 통합하여 도출하기 때문이다. 
이와 비슷한 이유로 단독의 Attention Function을 사용하는 것 보다 여러 개의 함수를 학습하는 것이 더 성능에 좋은 영향을 미쳐 
해당 연구에서는 8개의 Attention Function을 이용한 Multi-Head Attention을 사용하였다.
 
Masked Multi-Head Attention
Transformer 에서는 Auto Regressive즉 문장을 왼쪽부터 읽어서 다음 단어를 예측한다. 
따라서 Decoder의 부분에서 단어를 생성할 경우에 해당시점 i 다음의 단어의 데이터의 영향을 받는 것을 방지하고자 -inf의 값으로 해당 데이터를 가려두어 softma에 반영되지 못하도록 처리한다.
Applications of Attention in our Transformer
Transformer에서는 Attention을 다음과 같이 사용한다.
	Encoder- Decoder attention: Encoder 와 Decoder 의 Sequence의 Q,K, V를 통해 attention을 구한다.
	Encoder attention: Encoder의 input끼리의 attention을 구한다.
	Decoder attention: Decoder에서 input을 바탕으로 attention을 구하지만 다음 단어를 Masking하여 attention을 구한다.

3.2.3 Position-wise Feed Forward Networks
FFN(x)=max⁡(0,xW_1+b_1 ) W_2+b_2
Encoder와 Decoder내의 Layer를 Fully connected feed-forward 가 연결을 하고 있다. 각각의 위치에 적용되지만 가중치는 동일하게 때문에 이를 kernel size 가 1인 CNN의 연산으로 보아도 무방하다. 
이를 통해 얻어지는 결과는 기존의 Embedding에서 비선형성을 부여한다. 

3.2.4 Positional Encoding
recurrence와 convolution을 사용하지 않기 때문에 위치에 대한 정보를 embedding 과정에 포함할 필요가 있다. 즉 단어의 순서를 반영하기 위해 필요한 과정이다. 
순서를 부여하기 위해 주기를 가지는 삼각함수를 활용하여 순서에 대한 정보를 포함하도록 하였다.

3.3 Contribution
Transformer에서 기존의 순차적 모델을 계선하기 위해 attention에만 의존한 새로운 모델을 제안하였다. 이를 통해 Transformer가 얻은 효용은 다음과 같다.
	Layer당 계산의 복잡도 감소 및 계산의 총량 감소
	장기 종속성 문제 해결
이러한 발전을 바탕으로 Transformer는 해당 연구 시점 기계번역에서 SOTA를 달성하였다.

4 실험 및 결과

4.1 Dataset
WMT 2014 English-German dataset : 450만 문장 - 37,000 tokens
WMT 2014 English-Fraench dataset : 360만 문장 - 32,000 tokens

4.2 Hardware and Schedule
8 NVIDIA P100 GPUs
100,000 steps or 12 hours
300,000 steps or 3.5 days (big model)

4.3 Optimizer
Adam optimizer를 사용

4.4 Regularization
	Residual Dropout
	Label Smoothing
	
4.6. Results
 
Transformer는 WMT 2014 English-to-German translation task와 WMT 2014 English-to-Franch translation task에서 기존 모델들 대비 가장 좋은 BELU score를 기록하였다.
4.7. Conclusion
Transformer를 제안함으로 recurrence가 일어나던 기존의 모델과 다르게 순환구조 그리고 컨볼루션을 attention로 대체하여 계산량을 줄이고 더 좋은 성능을 얻었다. 
해당 연구를 기점으로 이후의 언어 모델들은 Transformer의 영향을 받게 되었다.
</pre>

# Language Models are Unsupervised Multitask Learners 리뷰

<pre>
1. Introduction 
Pre-training 이후 fine-tuning이라는 과정을 통해 많은 task를 해결하는 방법이 제안되어 한 개의 모델이 여러 개의 task를 수행할 수 있게 되었지만 여전히 task별로 훈련이 필요한 상황이다.
이에 해당 논문에서는 많은 데이터와 큰 모델을 통해 fine-tuning없이 바로 모든 task에 적용이 가능한 zero-shot으로 좋은 성능을 낼 수 있다는 것을 GPT-2를 통해서 보이고 있다.

1. 1 논문이 다루는 task 와 기존 연구 
해당 논문 에서는 머신러닝이 빅데이터와 큰 용량의 모델 그리고 지도학습을 사용하여 아주 우수하게 많은 task를 수행하고 있는 점을 주목하였다. 
하지만 이러한 과정에서 데이터의 분포에 따라 매우 민감하게 반응하며 대부분의 시스템들은 좁은 범위에 task에 우수하며 일반화하기에는 부족하다는 것을 지적하였다.
사람이 언어를 알면 대화가 가능하고 문서를 요약하는 등 언어 기반의 많은 문제를 해결할 수 있듯이 해당 논문에서도 많은 도메인에 일반화가 가능한 모델을 제안하고자 한다.

기존 연구
Multitask Learning
이는 일반화를 위한 프레임 워크이다. 하지만 NLP에 관해서는 아직 초기 단계이다. 현재의 기계학습에서는 매우 많은 데이터가 일반화를 위해 필요하다. 
그렇기에 Multitask training을 위해서도 많은 예제가 필요한 것이 당연해 보인다. 이를 활용하기 위해 적용이 가능한 데이터 셋을 만드는 것은 현재로서는 매우 어려운 일로 간주된다

Pre-training & supervised fine-tuning
BERT와 같이 Pre-training 이후 task에 맞추어 supervised fine-tuning는 현재 가장 좋은 성능을 보이고 있지만 여전히 task별로 학습을 필요로 하고 있다.
해당논문은 task를 위한 지도학습을 최소한으로 사용하거나 사용하지 않을 경우 다양한 task에서 더 발전을 기대할 수 있을 것이라고 생각하고 있다. 
따라서 새롭게 제안하는 방법으로 Language 모델의 변화 없이 다양한 downstream task에 zero-shot setting을 기반으로 접근하는 방법을 보이고 있다.

<pre>
2. Approach
 해당 연구의 중심이 되는 것은 Language 모델이다. 언어는 순서가 있기 때문에 이를 결합확률로 접근하는 것은 일반적인 방법이다.
p(x)= ∏_i^n▒〖p(s_n |s_1,⋯,s_(n-1))〗
 Transformer에서 attention을 통해 새롭게 제시한 구조를 통해 Auto Regressive 한 방법은 더욱 발전을 이루게 되었다.
하나의 task에 대해 수행하는 모델을 만드는 경우에 조건부확률 분포 p(output | input) 으로 나타낼 수 있으며 
여러 개에 task에 범용적으로 수행이 가능한 모델을 만들 경우에는 p(output | input, task) 으로 나타내야 한다. 
기존 모델의 경우 Encoder나 Decoder를 특정 task에 맞춤으로 설계한다. 
하지만 LM을 통해 전반적으로 모든 task를 수행하기 위해 해당 연구에서는 task에 따라 구조를 바꾸지 않고 task를 수행하는 방법을 제안하고 있다.

2.1 Training Dataset
이전에 제안된 방법들 에서는 모델을 학습시키기 위해 한가지 분야의 데이터들을 사용하였다. 
하지만 해당 연구에서는 일반화 성능이 필요하기 때문에 많은 분야의 데이터들이 필요하다. 
이를 위해 사용할 수 있는 데이터로는 Common Crawl과 같은 데이터가 있으나 더 좋은 품질의 데이터를 사용하기 위해 직접 데이터를 수집하여 사용하였다. 
Reddit에서 기준을 설정한 이후 데이터를 수집하였고 그 결과 40GB, 8000만개 이상의 데이터 셋을 구축할 수 있게 되었다.

2.2 Input Representation
Byte Pair Encoding (BPE)
해당 연구에서는 Representation을 위해 BPE를 사용하였다. BPE는 Fasttext와 같이 subword 기반의 인코딩 방법이다. 
문자 단위로 단어를 분리하고 빈도수가 높은 문자쌍을 지속적으로 Vocab에 추가하는 방법이다. 예를 들자면 
〖Vocab〗_word={apple,avaliable,caable}
〖Vocab〗_char={a,p,l,e,v,i,b,c,p}
〖Vocab〗_BPE={a,p,l,e,v,i,b,c,p,le,ble,able}
〖Vocab〗_word,〖Vocab〗_char,〖Vocab〗_BPE 순서로 단어 수준에서 문자 수준의 데이터를 얻은 이후 계속 반복하여 자주 등장하는 문자쌍을 추가하는 방법이다[1].

2.3 Model
GPT-2는 Transformer기반의 구조이다. Transformer의 decoder를 활용하였으며 기본적으로 GPT-1과 동일하다.
GPT-1의 경우 각각 테스크별로 입력을 다르게 만들어서 학습을 진행시키며, pre-training을 진행한 모델을 이용하면, fine-tuning시의 데이터가 적더라도 좋은 성능을 보여주었다[3].
<pre>

<pre>
3. 실험 및 결과	
3.1 Language Modeling
PBE를 통해 Byte단위로 작동하기 때문에 별도의 전처리나 tokenization이 필요하지 않다. 이로 인해 모든 벤치마크 데이터 셋에 적용이 가능하다는 강점이 생긴다
PBE로 Web Text를 처리할 경우 400억 byte 중에 26개만이 [UNK]로 처리되어 뛰어난 성능을 보인다. 
Web Tex를 통해 훈련된 결과는 많은 도메인과 데이터 셋에 잘 작동하였고 , 8개 중에서 7개의 데이터 셋에서 SOTA를 달성하였다. 
 
1BW에 대해서 좋지 않은 성능이 나타난 것은 해당 데이터 셋에서 문장을 섞어두어 긴 문장 구조가 다 사라졌기 때문이라고 해당 논문은 분석하고 있다.

3.2 Children’s Book Test
Children’s Book Test 에서는 단어를 구별하는 목적으로 만들어진 데이터 셋을 활용한다.  단어의 품사 또는 다음 단어를 예측하는 방식으로 모델의 성능을 평가한다.
해당 연구를 통해 모델의 크기가 커질수록 성능이 사람의 수준으로 수렴해가고 있는 것을 확인할 수 있다.


3.4. Winograd Schema Challenge
Winograd Schemas Challenge는 자연어 문장에서 특정 대명사가 어떤 대상을 지칭하는지를 판별하는 고전적인 자연어처리 문제로, 문법적으로는 모호하지만 의미적으로는 사람에게 명확한 문제이다[4]. 
따라서 해당 논문에서는 의미가 명확하지 않은 단어가 주어졌을 때 이를 파악하는 능력을 알기 위해 해당 Challenge를 수행하고 성능을 평가하였다. 
데이터가 273개만 주어져 그 크기가 작음에도 불구하고 높은 성능을 보였다.
이를 통해 GPT-2는 언어를 더 잘 이해하여서 모호하지만 사람은 알 수 있는 문장을 해석하는 능력을 갖췄다고 생각 해 볼 수 있다.

3.5 SOTA를 달성하지 못한 task
요약과제 기계 번역 , Question and Answering 에서는 많은 모델들보다 성능이 좋지 않은 모습을 찾을 수 있었다.

3.6 Generalization & Memorization
Test 데이터 셋과 train데이터 셋에 중복이 많은 상태에서 일반화 능력 검증을 시도할 경우 이미 아는 test 데이터 셋을 그저 다시 출력하는 Memorization으로 볼 수 있다. 
따라서 해당 논문에서는 데이터 셋의 중복을 표기하였다.추가로 모델 용량에 따른 Perplexity 그래프를 제공하였다.
</pre>

<pre>
4. Conclusion
BERT와 Transformer와 비교하였을 때 GPT-2에서 특이하게 보였던 부분은 모델의 구조를 설명하는 부분보다 
어떻게 데이터를 수집하고 데이터의 중복을 해결할지 등 자연어 데이터에 대한 관심을 많이 기울인 점이였다. 
언어를 이해하는 방법에 대해 많은 그리고 좋은 데이터를 통해 학습시키는 방법을 제안한 부분도 논문의 중요한 부분이라고 생각한다.
해당 논문에서 가장 중요한 것으로 꼽히는 부분은 zero-shot, 즉 언어를 잘 이해하는 모델을 변형 없이 여러가지 task에 적용에 대한 가능성을 보여준 점이다. 
요약 등의 task에서는 baseline에 근접한 성능을 보여주는 듯 아쉬운 능을 보여주었고 논문에서도 직접적으로 zero-shot 자체는 아직 사용되기에 부족하다고 언급하였다. 
하지만 여전히 여러 task에 대해서 SOTA를 달성한 점과 모델의 크기가 커질 때 마다 성능이 증가한다는 것을 보여준 것에서 앞으로의 발전 가능성을 크게 보여주었다고 생각한다.
</pre>


<pre>
참고문헌
1.	“[논문 리뷰, GPT-2]Language Models are Unsupervised Multitask Learners”. Deeppago’s study note, 2022년 5월 5일, https://deeppago.tistory.com/127.
2.	“GPT-2 논문 정리(논문 리뷰) - Language Models are Unsupervised Multitask Learners paper review”. 꿈 많은 사람의 이야기, 2021년 8월 30일, https://lsjsj92.tistory.com/620.
3.	ActionPower. “[액션파워LAB] GPT-1(Generative Pre-Training) 알아보기”. Medium, 2023년 6월 2일, 
https://actionpower.medium.com/%EC%95%A1%EC%85%98%ED%8C%8C%EC%9B%8Clab-gpt-1-generative-pre-training-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0-f63f18efa625.
4.	Python, Machine & Deep Learning. https://greeksharifa.github.io/. accessed in 2024.5.30.
5.	“[NLP] 언어모델의 평가지표 ‘Perplexity’ 개념 및 계산방법”. Hey Tech, 2022년 4월 5일, https://heytech.tistory.com/344.
</pre>